In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

# Fonction pour extraire le titre du produit
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id": "productTitle"})
        return title.text.strip() if title else ""
    except AttributeError:
        return ""

# Fonction pour extraire le prix du produit
def get_price(soup):
    try:
        price = soup.find("span", attrs={'id': 'priceblock_ourprice'})
        if price:
            return price.text.strip()
        
        price = soup.find("span", attrs={'id': 'priceblock_dealprice'})
        if price:
            return price.text.strip()

        price_whole = soup.find("span", attrs={"class": "a-price-whole"})
        price_fraction = soup.find("span", attrs={"class": "a-price-fraction"})

        if price_whole and price_fraction:
            return f"{price_whole.text.strip()}.{price_fraction.text.strip()}"
        
    except AttributeError:
        return ""
    
    return ""

# Fonction pour extraire la note du produit
def get_rating(soup):
    try:
        rating = soup.find("span", attrs={'class': 'a-icon-alt'})
        return rating.text.strip() if rating else ""
    except AttributeError:
        return ""

# Fonction pour extraire le nombre d'avis clients
def get_review_count(soup):
    try:
        reviews = soup.find("span", attrs={'id': 'acrCustomerReviewText'})
        return reviews.text.strip() if reviews else ""
    except AttributeError:
        return ""

# Fonction pour extraire la disponibilité du produit
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id': 'availability'})
        return available.find("span").text.strip() if available else "Not Available"
    except AttributeError:
        return "Not Available"
def get_products(url):
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }

    # Stockage des données
    product_data = {"title": [], "price": [], "rating": [], "reviews": [], "availability": []}

    while url:
        response = requests.get(url, headers=HEADERS)
        
        if response.status_code != 200:
            print(f"⚠ Erreur {response.status_code} - Impossible de charger {url}")
            break
        
        soup = BeautifulSoup(response.content, "html.parser")

        # Extraire les liens des produits
        links_list = []
        for link in soup.find_all("a", attrs={'class': 'a-link-normal s-no-outline'}):
            product_link = link.get('href')
            if product_link:
                links_list.append("https://www.amazon.com" + product_link)

        # Extraire les détails de chaque produit
        for link in links_list:
            new_response = requests.get(link, headers=HEADERS)
            
            if new_response.status_code != 200:
                print(f"⚠ Skipping {link} (Status: {new_response.status_code})")
                continue
            
            new_soup = BeautifulSoup(new_response.content, "html.parser")

            product_data['title'].append(get_title(new_soup))
            product_data['price'].append(get_price(new_soup))
            product_data['rating'].append(get_rating(new_soup))
            product_data['reviews'].append(get_review_count(new_soup))
            product_data['availability'].append(get_availability(new_soup))

            time.sleep(1)  # Petite pause pour éviter d’être bloqué

        # Trouver le bouton "Next" pour passer à la page suivante
        next_page = soup.find("li", attrs={"class": "a-last"})
        if next_page and next_page.find("a"):
            url = "https://www.amazon.com" + next_page.find("a")["href"]
            print(f"➡️ Passer à la page suivante : {url}")
        else:
            print("✅ Aucune page suivante trouvée. Fin du scraping.")
            break

    return product_data

In [38]:
if __name__ == '__main__':
    start_url = "https://www.amazon.com/s?k=beauty+and+cosmetics"
    
    # Récupérer tous les produits en parcourant toutes les pages
    all_products = get_products(start_url)

    # Convertir en DataFrame
    amazon_df = pd.DataFrame(all_products)

    # Supprimer les entrées sans titre
    amazon_df.replace('', np.nan, inplace=True)
    amazon_df.dropna(subset=['title'], inplace=True)

    # Sauvegarder en CSV
    amazon_df.to_csv("amazon_all_products.csv", header=True, index=False)

    print("✅ Tous les produits ont été enregistrés dans amazon_all_products.csv")

✅ Aucune page suivante trouvée. Fin du scraping.
✅ Tous les produits ont été enregistrés dans amazon_all_products.csv


In [34]:
amazon_df

,title,price,rating,reviews,availability
0,Sony Playstation PS4 1TB Black Console,199..75,4.6 out of 5 stars,"1,602 ratings",Only 1 left in stock - order soon.
1,PlayStation 4 Slim 1TB Console,369..99,4.7 out of 5 stars,"15,789 ratings",In Stock
2,Sony PlayStation 4 Slim 1TB Premium Bundle (Re...,249..90,3.2 out of 5 stars,7 ratings,In Stock
3,PlayStation®4 Console – Call of Duty® Modern W...,379..92,4.4 out of 5 stars,143 ratings,In Stock
4,"Playstation Sony 4, 500GB Slim System [CUH-221...",189..99,4.4 out of 5 stars,383 ratings,Only 1 left in stock - order soon.
5,Sony PlayStation 4 Console 1TB - Black (Renewed),224..99,3.2 out of 5 stars,20 ratings,Only 3 left in stock - order soon.
6,"Wall Mount for PS4 Slim,Steel PS4 Slim Mount I...",24..99,4.8 out of 5 stars,7 ratings,In Stock
7,PlayStation 4 Slim 1TB Console,249..98,4.1 out of 5 stars,318 ratings,
8,Sony PlayStation 4 Pro 1TB Console - Black (PS...,184..99,4.4 out of 5 stars,"4,139 ratings",Only 3 left in stock - order soon.
9,PlayStation 4,149..99,4.6 out of 5 stars,"13,652 ratings",Only 3 left in stock - order soon.
